# Extract, Load and Transform Data

## Objectives

Extract and prepare the Kaggle Stress Level Prediction dataset for analysis and modelling. Specifically:

- Validate dataset structure.
- Perform exploratory quality checks (missing values, duplicates, ranges).
- Clean and standardise data.
- Engineer features to improve interpretability and model performance.

## Inputs

- Raw Kaggle Stress Level Prediction Dataset `data/raw/stress_detection_data.csv` (originally from https://www.kaggle.com/datasets/shijo96john/stress-level-prediction)

## What this notebook does

1. Load and inspect the raw dataset.
2. Conduct data quality checks (missing values, duplicates, range validation).
3. Clean and standardise variables:
   - Convert binary Yes/No features to integer-encoded (0/1).
   - Parse `Bed_Time` and `Wake_Up_Time` strings into numeric format (minutes since midnight).
   - Standardise categorical variables.
4. Explore continuous feature distributions and detect skew/outliers.
5. Perform feature engineering:
   - Calculate sleep duration from bed/wake times (as validation).
   - Create interaction features capturing potential stress risk patterns.
   - Bin continuous variables into categories where useful.
6. Export a cleaned, feature-enhanced dataset for use in modelling and EDA notebooks.

## Outputs

- Processed dataset to `data/processed/stress_data_processed.csv` for use in:
  - `02_EDA.ipynb` (Exploratory Data Analysis Notebook)
  - `03_Modelling.ipynb` (Modelling and Evaluation Notebook)

---

## Change working directory

I need to change the working directory from the current folder to its parent folder (required because the notebook is being run from inside the jupyter notebooks subfolder). In the code below, I change the working directory from its current folder to its parent folder.

- I access the current directory with `os.getcwd()`

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory.

- `os.path.dirname()` gets the parent directory
- `os.chdir()` defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

## Section 1: Data Load

### About the Dataset

This notebook works with the Stress Level Prediction dataset from Kaggle. The dataset contains self-reported lifestyle, sleep, and health data from individuals, with a target variable indicating their stress level.

**Dataset Details:**
- **Source:** Kaggle - "Stress Level Prediction" (shijo96john)
- **Rows / Columns:** 773 rows x 22 columns
- **Target:** `Stress_Detection` (Low / Medium / High)

**Feature Categories:**
- **Demographics:** Age, Gender, Marital_Status, Occupation
- **Sleep:** Sleep_Duration, Sleep_Quality, Bed_Time, Wake_Up_Time
- **Lifestyle:** Physical_Activity, Screen_Time, Caffeine_Intake, Alcohol_Intake, Smoking_Habit, Work_Hours, Travel_Time, Social_Interactions, Meditation_Practice, Exercise_Type
- **Health Indicators:** Blood_Pressure, Cholesterol_Level, Blood_Sugar_Level

### In this section:

1. Load the raw dataset from `data/raw/stress_detection_data.csv`
2. Conduct basic shape and structure checks
3. Verify target variable distribution

In [ ]:
# Load libraries and the raw dataset

# Core file handling libraries
from pathlib import Path  # load the pathlib library

# Data manipulation and analysis libraries
import numpy as np  # load the numpy library
import pandas as pd  # load the pandas library

# Data visualisation libraries
import matplotlib.pyplot as plt  # load the matplotlib library
import seaborn as sns  # load the seaborn library

# Set display options
pd.set_option('display.max_columns', None)  # show all columns

# Load the raw dataset
df = pd.read_csv('data/raw/stress_detection_data.csv')  # load the dataset

df.shape  # display the shape of the dataframe

In [ ]:
df.head()  # display the first 5 rows of the dataframe

In [ ]:
# Display target variable distribution
stress_counts = df['Stress_Detection'].value_counts()  # count stress levels
total_rows = df.shape[0]  # get total number of rows

print("Target Variable Distribution:")
print("-" * 40)
for level in stress_counts.index:
    count = stress_counts[level]
    pct = (count / total_rows) * 100
    print(f"{level}: {count} ({pct:.1f}%)")
print("-" * 40)
print(f"Total: {total_rows} rows")

---

## Section 2: Data Structure and Quality Check

Basic data and structure information checks for the raw dataset:

- Check dataframe shape
- Check data types
- Check for missing values
- Check for duplicates

**Expected Column Types:**

**Numeric Continuous:**
- Age, Sleep_Duration, Sleep_Quality, Physical_Activity, Screen_Time
- Caffeine_Intake, Alcohol_Intake, Work_Hours, Travel_Time, Social_Interactions
- Blood_Pressure, Cholesterol_Level, Blood_Sugar_Level

**Categorical:**
- Gender, Occupation, Marital_Status, Exercise_Type, Stress_Detection

**Binary (Yes/No):**
- Smoking_Habit, Meditation_Practice

**Time Strings (to be parsed):**
- Bed_Time, Wake_Up_Time

In [ ]:
# Basic structure of the dataframe and quality check

print("\nColumn information:")
print("=" * 70)

# Check data types, non-null counts and missing values and create summary table
summary = (
    pd.DataFrame({
        'Data Type': df.dtypes,  # data types
        'Non-Null Count': df.notnull().sum(),  # non-null counts
        'Missing Values': df.isnull().sum(),  # missing values
        'Missing %': (df.isna().mean() * 100).round(3),  # percent missing
        'Unique Values': df.nunique()  # unique values
    })
    .sort_values(by='Missing Values', ascending=False)  # sort by missing values
)

summary  # display the summary table

In [ ]:
# Check for duplicate rows
duplicate_rows = df.duplicated().sum()  # count duplicate rows
print(f"\nNumber of duplicate rows: {duplicate_rows}")

if duplicate_rows > 0:
    print(f"\nDuplicate rows will be removed in the cleaning step.")
else:
    print("No duplicate rows found - data is clean in this respect.")

---

## Section 3: Range Validation

Before analysis, key continuous variables are checked below to ensure the values are sensible and within expected limits. This step helps confirm there are no invalid or extreme values (such as negative durations or unrealistic health indicators) that could distort analysis results.

**Expected Ranges:**
- Age: 18-100 (reasonable adult range)
- Sleep_Duration: 0-24 hours
- Sleep_Quality: 1-5 scale
- Screen_Time: 0-24 hours
- Work_Hours: 0-24 hours per day
- Blood_Pressure: 80-200 (systolic)
- Cholesterol_Level: 100-400
- Blood_Sugar_Level: 50-300

In [ ]:
# Validate ranges are sensible for continuous variables
print("\nValue Range Validation (Continuous Variables):")
print("-" * 60)

cont_cols = [
    "Age", "Sleep_Duration", "Sleep_Quality", "Physical_Activity",
    "Screen_Time", "Caffeine_Intake", "Alcohol_Intake", "Work_Hours",
    "Travel_Time", "Social_Interactions", "Blood_Pressure",
    "Cholesterol_Level", "Blood_Sugar_Level"
]  # list of continuous variable columns

for col in cont_cols:
    min_val = df[col].min()  # calculate minimum value
    max_val = df[col].max()  # calculate maximum value
    print(f"{col:25s}: min={min_val:8.2f}, max={max_val:8.2f}")

In [ ]:
# Check for any negative values (should be none for most features)
print("\nNegative Value Counts (Continuous Variables):")
print("-" * 60)

negative_counts = (df[cont_cols] < 0).sum()  # count negative values

if negative_counts.sum() > 0:
    print(negative_counts[negative_counts > 0])  # display counts of negative values
else:
    print("All continuous variables have valid non-negative values")

---

## Section 4: Descriptive Statistics

### Aim

To get an initial sense of the dataset by reviewing averages, middle values, variation, and any obvious extremes before conducting more detailed plots.

### Approach

`df.describe()` was used to generate summary statistics for the numeric variables. Adding `include="all"` includes non-numeric (categorical and object) columns in the summary, providing a more complete overview of the dataset.

This summary displays key statistics such as the mean (average), median (middle point), standard deviation (spread of values), and percentiles (positions within the distribution).

In [ ]:
# Summary Stats for numeric variables
df.describe()  # display summary statistics for numeric variables

In [ ]:
# Summary Stats for categorical variables
df.describe(include='object')  # display summary statistics for categorical variables

---

## Section 5: Distribution and Outlier Checks

### Aim

To examine the continuous features to understand their distribution, scale, and the presence of extreme values. This will help decide whether transformations (such as log scaling) or outlier handling is needed later.

### Approach

- Plot histograms to view the shape and skew of each variable.
- Use boxplots to spot potential outliers.
- Apply the IQR (interquartile range) method to count possible outlier values numerically.

In [ ]:
# Histograms for key continuous variables
key_cont_cols = [
    "Sleep_Duration", "Sleep_Quality", "Screen_Time",
    "Work_Hours", "Physical_Activity", "Caffeine_Intake"
]

fig, axes = plt.subplots(2, 3, figsize=(14, 8))
axes = axes.flatten()

for i, col in enumerate(key_cont_cols):
    axes[i].hist(df[col], bins=20, edgecolor='black', alpha=0.7)
    axes[i].set_title(col)
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Frequency')

plt.suptitle("Distribution of Key Continuous Features", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Boxplots for continuous variables to check for outliers
fig, axes = plt.subplots(2, 3, figsize=(14, 8))
axes = axes.flatten()

for i, col in enumerate(key_cont_cols):
    axes[i].boxplot(df[col].dropna())
    axes[i].set_title(col)
    axes[i].set_ylabel(col)

plt.suptitle("Boxplots of Key Continuous Features", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# IQR method to identify outliers
print("\nOutlier Detection using IQR Method:")
print("-" * 60)

for col in cont_cols:
    Q1 = df[col].quantile(0.25)  # first quartile
    Q3 = df[col].quantile(0.75)  # third quartile
    IQR = Q3 - Q1  # interquartile range
    lower_bound = Q1 - 1.5 * IQR  # lower bound for outliers
    upper_bound = Q3 + 1.5 * IQR  # upper bound for outliers
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]  # identify outliers
    if len(outliers) > 0:
        print(f"{col:25s}: {len(outliers)} outliers (IQR method)")

---

## Section 6: Categorical and Binary Feature Checks

### Goal

Confirm that categorical and binary variables are correctly formatted and assess their distributions. This ensures data integrity and highlights any imbalance.

### Approach

- Use `.value_counts()` to check the frequency of categories.
- Create bar charts for visual confirmation.
- Convert Yes/No binary variables to integer (0/1) for modelling.

In [ ]:
# Categorical feature checks
cat_cols = ["Gender", "Marital_Status", "Exercise_Type", "Occupation"]

print("Categorical Feature Value Counts:")
print("=" * 60)

for col in cat_cols:
    print(f"\n{col}:")
    print(df[col].value_counts())

In [ ]:
# Binary feature checks (Yes/No)
binary_cols = ["Smoking_Habit", "Meditation_Practice"]

print("\nBinary Feature Value Counts (Yes/No):")
print("=" * 60)

for col in binary_cols:
    print(f"\n{col}:")
    print(df[col].value_counts(normalize=True).round(3))

In [ ]:
# Bar plots for categorical variables
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for i, col in enumerate(cat_cols):
    value_counts = df[col].value_counts()
    axes[i].bar(value_counts.index, value_counts.values, edgecolor='black')
    axes[i].set_title(f'{col} Distribution')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Count')
    axes[i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Time string checks (Bed_Time and Wake_Up_Time)
print("\nTime Variable Samples:")
print("=" * 60)
print("\nBed_Time unique values (first 10):")
print(df['Bed_Time'].unique()[:10])
print("\nWake_Up_Time unique values (first 10):")
print(df['Wake_Up_Time'].unique()[:10])

---

## Section 7: Correlation Analysis

Heatmap to explore relationships between numeric variables and see how strongly they correlate with each other. This helps identify redundancy between features and provides an early indication of predictive signal.

**Note:** Correlation with the target variable (`Stress_Detection`) will be explored after encoding it numerically.

In [ ]:
# Correlation Heatmap for numeric features
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
corr = df[numeric_cols].corr()  # calculate the correlation matrix

plt.figure(figsize=(12, 10))
sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', cbar=True, square=True)
plt.title("Correlation Heatmap - Numeric Features", fontsize=14)
plt.tight_layout()
plt.show()

---

## Section 8: Data Cleaning and Feature Engineering

### Goal

Clean the dataset and create new variables/transformations to capture stress patterns that are not easily visible in the raw features. This step improves model performance and interpretability.

### Approach

1. **Convert Yes/No binary features** to integer (0/1) encoding.
2. **Parse Bed_Time and Wake_Up_Time** strings into numeric format (minutes since midnight).
3. **Encode target variable** (`Stress_Detection`) ordinally (Low=0, Medium=1, High=2).
4. **Create interaction features** to capture combined effects.
5. **Bin continuous variables** into categories where useful for analysis.

In [ ]:
# Create a copy of the dataframe for processing
df_processed = df.copy()

# 1. Convert Yes/No binary features to integer (0/1)
binary_yes_no_cols = ["Smoking_Habit", "Meditation_Practice"]

for col in binary_yes_no_cols:
    df_processed[col] = df_processed[col].map({'Yes': 1, 'No': 0}).astype(int)

print("Binary features converted to 0/1:")
print(df_processed[binary_yes_no_cols].head())

In [ ]:
# 2. Parse Bed_Time and Wake_Up_Time to minutes since midnight

def time_to_minutes(time_str):
    """
    Convert time string (e.g., '10:00 PM', '7:00 AM') to minutes since midnight.
    """
    try:
        time_str = time_str.strip()
        # Parse the time using pandas
        time_obj = pd.to_datetime(time_str, format='%I:%M %p')
        return time_obj.hour * 60 + time_obj.minute
    except:
        return np.nan

# Apply the conversion
df_processed['Bed_Time_Minutes'] = df_processed['Bed_Time'].apply(time_to_minutes)
df_processed['Wake_Up_Time_Minutes'] = df_processed['Wake_Up_Time'].apply(time_to_minutes)

print("\nTime conversion results (first 5 rows):")
print(df_processed[['Bed_Time', 'Bed_Time_Minutes', 'Wake_Up_Time', 'Wake_Up_Time_Minutes']].head())

In [ ]:
# Verify time conversion ranges
print("\nBed_Time_Minutes range:")
print(f"Min: {df_processed['Bed_Time_Minutes'].min()}, Max: {df_processed['Bed_Time_Minutes'].max()}")
print("\nWake_Up_Time_Minutes range:")
print(f"Min: {df_processed['Wake_Up_Time_Minutes'].min()}, Max: {df_processed['Wake_Up_Time_Minutes'].max()}")

# Check for any NaN values introduced
print(f"\nNaN values in Bed_Time_Minutes: {df_processed['Bed_Time_Minutes'].isna().sum()}")
print(f"NaN values in Wake_Up_Time_Minutes: {df_processed['Wake_Up_Time_Minutes'].isna().sum()}")

In [ ]:
# 3. Encode target variable ordinally (Low=0, Medium=1, High=2)
stress_mapping = {'Low': 0, 'Medium': 1, 'High': 2}
df_processed['Stress_Level_Encoded'] = df_processed['Stress_Detection'].map(stress_mapping)

print("\nTarget variable encoding:")
print(df_processed[['Stress_Detection', 'Stress_Level_Encoded']].value_counts().sort_index())

In [ ]:
# 4. Create interaction features

# Sleep efficiency proxy: Sleep Quality relative to Sleep Duration
df_processed['Sleep_Efficiency'] = df_processed['Sleep_Quality'] / df_processed['Sleep_Duration']

# Screen time to physical activity ratio
df_processed['Screen_Activity_Ratio'] = df_processed['Screen_Time'] / (df_processed['Physical_Activity'] + 0.1)  # add small value to avoid division by zero

# Work-Life balance indicator: Work_Hours + Travel_Time
df_processed['Work_Travel_Total'] = df_processed['Work_Hours'] + df_processed['Travel_Time']

# High screen time flag (above median)
df_processed['High_Screen_Time'] = (df_processed['Screen_Time'] > df_processed['Screen_Time'].median()).astype(int)

# Low sleep flag (below 6 hours)
df_processed['Low_Sleep'] = (df_processed['Sleep_Duration'] < 6).astype(int)

print("\nInteraction features created:")
print(df_processed[['Sleep_Efficiency', 'Screen_Activity_Ratio', 'Work_Travel_Total', 'High_Screen_Time', 'Low_Sleep']].describe())

In [ ]:
# 5. Bin continuous variables into categories

# Sleep Duration bins
df_processed['Sleep_Duration_Bin'] = pd.cut(
    df_processed['Sleep_Duration'],
    bins=[0, 5, 6, 7, 8, 24],
    labels=['Very Low (<5h)', 'Low (5-6h)', 'Normal (6-7h)', 'Good (7-8h)', 'High (>8h)']
)

# Work Hours bins
df_processed['Work_Hours_Bin'] = pd.cut(
    df_processed['Work_Hours'],
    bins=[0, 6, 8, 10, 24],
    labels=['Low (<6h)', 'Normal (6-8h)', 'High (8-10h)', 'Very High (>10h)']
)

# Screen Time bins
df_processed['Screen_Time_Bin'] = pd.cut(
    df_processed['Screen_Time'],
    bins=[0, 2, 4, 6, 24],
    labels=['Low (<2h)', 'Moderate (2-4h)', 'High (4-6h)', 'Very High (>6h)']
)

print("\nBinned variable distributions:")
print("\nSleep Duration Bins:")
print(df_processed['Sleep_Duration_Bin'].value_counts())
print("\nWork Hours Bins:")
print(df_processed['Work_Hours_Bin'].value_counts())
print("\nScreen Time Bins:")
print(df_processed['Screen_Time_Bin'].value_counts())

In [ ]:
# Verify all engineered features were created correctly

print("Full feature set verification")
print("=" * 70)

# List ALL engineered features
engineered_features = [
    "Bed_Time_Minutes",
    "Wake_Up_Time_Minutes",
    "Stress_Level_Encoded",
    "Sleep_Efficiency",
    "Screen_Activity_Ratio",
    "Work_Travel_Total",
    "High_Screen_Time",
    "Low_Sleep",
    "Sleep_Duration_Bin",
    "Work_Hours_Bin",
    "Screen_Time_Bin"
]

# 1. Check if all features exist
print("\nFeature existence check:")
missing_features = [f for f in engineered_features if f not in df_processed.columns]

if missing_features:
    print(f"Missing features: {missing_features}")
else:
    print(f"All {len(engineered_features)} engineered features created successfully")

# 2. Feature count summary
print(f"\nFeature count summary:")
original_features = 22
new_features = len(engineered_features)
total_features = df_processed.shape[1]

print(f"Original features: {original_features}")
print(f"Engineered features: {new_features}")
print(f"Total features in dataset: {total_features}")

# 3. Check for NaN values in engineered features
print(f"\nNaN check in engineered features:")
nan_counts = df_processed[engineered_features].isna().sum()

if nan_counts.sum() > 0:
    print(f"NaN values found:")
    print(nan_counts[nan_counts > 0])
else:
    print(f"No NaN values in engineered features")

# 4. Data type verification
print(f"\nDatatype verification for engineered features:")
for feat in engineered_features:
    dtype = df_processed[feat].dtype
    print(f"{feat:30s}: {dtype}")

print("\n" + "=" * 70)
print("Feature engineering verification complete")

---

## Section 9: Save Processed Dataset

Save the cleaned and feature-engineered dataset so it can be reused in modelling and visualisation notebooks without re-running the ETL pipeline.

In [ ]:
# Create processed folder if it does not exist
processed_path = Path("data/processed")  # define the path for the processed data folder
processed_path.mkdir(parents=True, exist_ok=True)  # create the folder if it does not exist

# Define output file path
output_file = processed_path / "stress_data_processed.csv"  # define the output file path

# Save processed file to CSV
df_processed.to_csv(output_file, index=False)  # save the dataframe to a CSV file without the index

print(f"Processed file saved to {output_file}")
print(f"Final Shape: {df_processed.shape}")

# Verify file exists and print confirmation
if output_file.exists():
    print(f"File exists: {output_file}")

In [ ]:
# Display final column list for reference
print("\nFinal Dataset Columns:")
print("=" * 70)
for i, col in enumerate(df_processed.columns, 1):
    print(f"{i:2d}. {col}")

---

## Section 10: Summary and Next Steps

### Summary of work completed in this notebook:

1. **Loaded** the raw Stress Level Prediction dataset from `data/raw/stress_detection_data.csv` and confirmed structure (773 rows x 22 columns).

2. **Verified data quality:**
   - No missing values detected
   - No duplicate rows found
   - All continuous variables have sensible ranges

3. **Confirmed target variable distribution:**
   - Medium: 310 (40.1%)
   - High: 301 (38.9%)
   - Low: 162 (21.0%)

4. **Explored distributions:**
   - Examined continuous variable distributions via histograms and boxplots
   - Identified potential outliers using IQR method
   - Checked categorical and binary feature distributions

5. **Cleaned and transformed data:**
   - Converted Yes/No binary features to integer (0/1)
   - Parsed Bed_Time and Wake_Up_Time strings to minutes since midnight
   - Encoded target variable ordinally (Low=0, Medium=1, High=2)

6. **Engineered new features:**
   - Sleep_Efficiency (Sleep_Quality / Sleep_Duration)
   - Screen_Activity_Ratio (Screen_Time / Physical_Activity)
   - Work_Travel_Total (Work_Hours + Travel_Time)
   - High_Screen_Time flag (above median)
   - Low_Sleep flag (below 6 hours)
   - Binned variables: Sleep_Duration_Bin, Work_Hours_Bin, Screen_Time_Bin

7. **Exported** the processed dataset (773 rows) to `data/processed/stress_data_processed.csv`.

### Insights from ETL & Feature Engineering:

- The target variable is fairly balanced across the three stress levels, with a slight skew towards Medium and High stress.
- Sleep duration and quality show variation that may be associated with stress levels.
- Screen time and work hours show reasonable distributions with some high values.
- Binary features (Smoking_Habit, Meditation_Practice) are now properly encoded for modelling.
- Time features (Bed_Time, Wake_Up_Time) are now in numeric format for analysis.

### Next Steps:

1. **Exploratory Data Analysis (02_EDA.ipynb):**
   - Conduct hypothesis tests (H1-H8) as outlined in the README
   - Visualise relationships between features and stress levels
   - Calculate statistical significance and effect sizes

2. **Modelling (03_Modelling.ipynb):**
   - Train baseline models (Logistic Regression, Decision Tree)
   - Evaluate using Accuracy, Macro F1, confusion matrix
   - Feature importance analysis
   - Model comparison and selection